# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. В начале исследования будет произведен обзор данных, выявлены и устранены возможные пропуски, дубликаты и аномалии. Затем данные будут категоризированы и представлены в сводных таблицах для получения ответов на поставленные вопросы.  

### Шаг 1. Обзор данных.

Осуществляется загрузка файла и первичное знакомство с данными.

In [1]:
import pandas as pd #импортируем библиотеку pandas

data = pd.read_csv('/datasets/data.csv') #считываем csv-файл

data.info() #смотрим сводку основной информации
data.head() #смотрим как отображаются данные

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Первичный обзор данных показывает, что в двух столбцах - days_employed и total_income - есть пропуски, кроме того в столбце days_employed тажже наблюдаются аномальные (отрицательные) значения.

### Шаг 2.1 Заполнение пропусков.

Далее определяется доля пропущенных значений в каждом из столбцов.

In [2]:
print(data['days_employed'].isna().sum()/len(data)) #найдем долю пропущенных значений в столбце days_employed
print(data['total_income'].isna().sum()/len(data)) #найдем долю пропущенных значений в столбце total_income

0.10099883855981417
0.10099883855981417


Доли совпадают до последних знаков. Это наводит на предположение о том, что пропуски в обоих столбцах приходятся на одни и те же строки. Требуется дополнительная проверка.

In [3]:
data[(data['days_employed'].isna()) | (data['total_income'].isna())] #выводим строки с пропусками

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Действительно, похоже, что данные отсутствуют в одних и тех же строках, что может говорить об ошибках записи в выгрузке или о том, что в какой-то части базы данных не велась подобная статистика. 

Осуществляется замена пропусков в столбце total_income на медианные значения.

In [4]:
income_medians = data.groupby('income_type')['total_income'].median() #рассчитываем медианные значения total_income в группировке по income_type

display(income_medians) #посмотрим на медианные значения

income_types = data['income_type'].unique() #создаем список с уникальными наименованиями из столбца income_type

for income_type in income_types: #перебираем категории income_type
    data.loc[((data['income_type'] == income_type) & (data['total_income'].isna())), 'total_income'] = income_medians[income_type] #заменяем пропуски в столбце total_income для выбранной категории income_type на медианные значения

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Медианные значения лучше подойдут для замены пропусков, так как в большей степени отражают "типичные" показатели в выборке при большом разбросе значений количественных переменных.

### Шаг 2.2 Проверка данных на аномалии и исправления.

Прежде чем заполнять пропуска в столбце days_employed следует внимательнее ознакомиться с аномальными значениями и устранить их.

In [5]:
data[data['days_employed'] < 0] #выводим строки с отрицательными значениями

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Аномалии присутсвуют более чем в 15000 строк, что явно говорит о системном характере ошибки (вероятно, произошла смена знака где-то в процессе обработки данных). 

Осуществляется смена знака для отрицательных значений в столбце days_employed.

In [6]:
data['days_employed'] = data['days_employed'].abs() #меняем знак отрицательных значений в столбце days_employed 

Осуществляется замена пропусков в столбце days_employed на медианные значения.

In [7]:
days_employed_median = data['days_employed'].median() #рассчитываем медианное значение days_employed

data['days_employed'] = data['days_employed'].fillna(days_employed_median) #заменяем пропуски в столбце days_employed на медианное значение

В ходе дальнейшего рассмотрения данных аналогичная проблема с отрицательными значениями была выявлена для столбца children.

Осуществляется смена знака для отрицательных значений в столбце children.

In [8]:
data['children'] = data['children'].abs() #меняем знак отрицательных значений в столбце children 

Также столбцы days_employed и total_income оцениваются с точки зрения наличия иных числовых аномалий.

##### Столбец days_employed:

In [9]:
data['days_employed'].describe() #смотрим описание значений в столбце days_employed

count     21525.000000
mean      60378.032733
std      133257.558514
min          24.141633
25%        1025.608174
50%        2194.220567
75%        4779.587738
max      401755.400475
Name: days_employed, dtype: float64

Максимальное значение в столбце days_employed составляет 401755 дней, что примерно соответствует 1100 лет. Очевидно, что такое значение является невозможным. В связи с этим предлагается также заменить все неправдоподобно большие значения days_employed на медианные (за пороговую величину будет принято 100 лет).

In [10]:
data.loc[data['days_employed'] > 36500, 'days_employed'] = days_employed_median #заменяем все значения больше 36500 дней в столбце days_employed на медианное значение

data['days_employed'].max() #посмотрим новый максимум days_employed

18388.949900568383

Новый максимум days_employed выглядит нормальным.

##### Столбец total_income:

In [11]:
data['total_income'].describe() #смотрим описание значений в столбце total_income

count    2.152500e+04
mean     1.652253e+05
std      9.804367e+04
min      2.066726e+04
25%      1.077982e+05
50%      1.425944e+05
75%      1.955499e+05
max      2.265604e+06
Name: total_income, dtype: float64

Значения в столбце total_income выглядят нормальными.

Проводится проверка внесенных изменений.

In [12]:
data.info() #смотрим сводку основной информации
data.head() #смотрим как отображаются данные

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,2194.220567,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Пропуски и аномальные значения больше не выявляются.

### Шаг 2.3. Изменение типов данных.

Осуществляется изменение типа данных в столбце total_income на целочисленный.

In [13]:
data['total_income'] = data['total_income'].astype('int') #заменяем тип данных на int в столбце total_income
data['days_employed'] = data['days_employed'].astype('int') #заменяем тип данных на int в столбце days_employed

Проведение проверки будет выполнено в ходе дальнейших шагов.

### Шаг 2.4. Удаление дубликатов.

Далее определяется наличие и количество строк-дубликатов.

In [14]:
print(data.duplicated().sum()) #найдем количество строк-дубликатов

54


Осуществляется удаление строк-дубликатов.

In [15]:
data = data.drop_duplicates() #удаляем строки-дубликаты

Затем проводится проверка на неявные дубликаты по остальным столбцам. Для этого выводятся уникальные значения по каждому столбцу и, в случае выявления неочевидных результатов, определяется их количество - для понимания того, носят ли подобные результаты разовый или повторяющийся характер. 

##### Столбец education:

In [16]:
print(data['education'].unique()) #выводим уникальные значения столбца education

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


Значения в столбце education записаны в разных регистрах, что требует исправления.

In [17]:
data['education'] = data['education'].str.lower() #приводим все значения в столбце education к нижнему регистру

Смена регистра позволит избавиться от неявных дубликатов в столбце education.

##### Столбец children:

In [18]:
print(data['children'].unique()) #выводим уникальные значения столбца children
data['children'].value_counts() #выводим количество уникальных значений столбца children

[ 1  0  3  2  4 20  5]


0     14107
1      4856
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

В столбце children встречается значение 20, при этом оно встречается 76 раз, что говорит не об уникальном случае, а о каком-то регулярном сбое (можно предположить, что это неверное отображение числа 2, полученное, например, из изначального вида "2.0").

Осуществляется замена значений в столбце children.

In [19]:
data['children'] = data['children'].replace(20, 2) #заменяем значения 20 на 2 в столбце children

Такая замена позволит сделать более разумные выводы по итогам исследования.

##### Столбец dob_years:

In [20]:
print(data['dob_years'].unique()) #выводим уникальные значения столбца dob_years

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]


В столбце dob_years встречается значение 0, но так как указанный столбец не критичен для итоговых вывовод исследования, на данном этапе можно относиться к подобным значениям, как к отсутствующей информации.

##### Столбец family_status:

In [21]:
print(data['family_status'].unique()) #выводим уникальные значения столбца family_status

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


Значения в столбце family_status записаны в разных регистрах, что требует исправления.

In [22]:
data['family_status'] = data['family_status'].str.lower() #приводим все значения в столбце family_status к нижнему регистру

Смена регистра позволит добиться лучшего визуального отображения данных в столбце family_status.

##### Столбец gender:

In [23]:
print(data['gender'].unique()) #выводим уникальные значения столбца gender
print(data['gender'].value_counts()) #выводим количество уникальных значений столбца gender

['F' 'M' 'XNA']
F      14189
M       7281
XNA        1
Name: gender, dtype: int64


В столбце gender встречается значение XNA, но так как указанный столбец не критичен для итоговых вывовод исследования, а подобное значение встречается всего один раз, на данном этапе его можно игнорировать.

##### Столбец income_type:

In [24]:
print(data['income_type'].unique()) #выводим уникальные значения столбца income_type

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Значения в столбце income_type выглядят нормальными.

##### Столбец debt:

In [25]:
print(data['debt'].unique()) #выводим уникальные значения столбца debt

[0 1]


Значения в столбце debt выглядят нормальными. Это бинарные значения, показывающие отсутствие или наличие долга.

##### Столбец purpose:

In [26]:
print(data['purpose'].unique()) #выводим уникальные значения столбца purpose

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Значения в столбце purpose преставляют собой разрозненный набор описаний, которые могут быть сведены к нескольким основным сущностям. Категоризация данных значений будет выполнена в ходе дальнейших шагов.

Повторно определяется наличие и количество строк-дубликатов после произведенных замен данных.

In [27]:
print(data.duplicated().sum()) #найдем количество строк-дубликатов

17


Осуществляется удаление появившихся строк-дубликатов.

In [28]:
data = data.drop_duplicates() #удаляем строки-дубликаты

Проводится проверка внесенных изменений.

In [29]:
data.info() #смотрим сводку основной информации

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int64 
 11  purpose           21454 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


Количество строк уменьшилось за счет удаленных дубликатов, а тип столбцов total_income и days_employed был корректно изменен.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Формируется и выводится датафрейм-словарь education.

In [30]:
data_dict_education = data[['education_id', 'education']] #выбираем столбцы education и education_id исходного датафрейма

data_dict_education = data_dict_education.drop_duplicates().reset_index(drop=True) #удаляем дубликаты с переназначением индексов 

data_dict_education.head() #смотрим как отображаются данные

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Формируется и выводится датафрейм-словарь family_status.

In [31]:
data_dict_family = data[['family_status_id', 'family_status']] #выбираем столбцы family_status и family_status_id исходного датафрейма

data_dict_family = data_dict_family.drop_duplicates().reset_index(drop=True) #удаляем дубликаты с переназначением индексов

data_dict_family.head() #смотрим как отображаются данные

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Столбцы education и family_status удаляются из сходного датафрейма.

In [32]:
data = data.drop(['education', 'family_status'], axis=1) #убираем столбцы education и family_status

data.head() #смотрим как отображаются данные

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,2194,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


Декомпозиция датафрейма прошла успешо.

### Шаг 2.6. Категоризация дохода.

Создается функция для определения категории дохода.

In [33]:
def total_income_category(total_income): #функция возвращает категорию в зависимости от величины дохода
    if total_income <= 30000: 
        return 'E'
    elif total_income <= 50000:
        return 'D'
    elif total_income <= 200000:
        return 'C'
    elif total_income <= 1000000:
        return 'B'
    else:
        return 'A'

Создается новый столбец total_income_category с категорией дохода, определенной на основе функции.

In [34]:
data['total_income_category'] = data['total_income'].apply(total_income_category) #применяем функцию total_income_category к столбцу total_income

Проводится проверка значений нового столбца total_income_category.

In [35]:
print(data['total_income_category'].unique()) #выводим уникальные значения столбца total_income_category

['B' 'C' 'D' 'E' 'A']


Все значения соответствуют заданным в функции.

Проводится проверка внесенных изменений.

In [36]:
data.info() #смотрим сводку основной информации
data.head() #смотрим как отображаются данные

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21454 non-null  int64 
 1   days_employed          21454 non-null  int64 
 2   dob_years              21454 non-null  int64 
 3   education_id           21454 non-null  int64 
 4   family_status_id       21454 non-null  int64 
 5   gender                 21454 non-null  object
 6   income_type            21454 non-null  object
 7   debt                   21454 non-null  int64 
 8   total_income           21454 non-null  int64 
 9   purpose                21454 non-null  object
 10  total_income_category  21454 non-null  object
dtypes: int64(7), object(4)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,2194,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


Категоризация дохода прошла успешно.

### Шаг 2.7. Категоризация целей кредита.

Создается функция для определения категории цели кредита.

In [37]:
def purpose_category(purpose): #функция возвращает категорию в зависимости от указанной цели
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'жил' in purpose or 'недвиж' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образ' in purpose:
        return 'получение образования'
    else:
        return 'прочее'

Создается новый столбец purpose_category с категорией цели кредита, определенной на основе функции.

In [38]:
data['purpose_category'] = data['purpose'].apply(purpose_category) #применяем функцию purpose_category к столбцу purpose

Проводится проверка значений нового столбца purpose_category.

In [39]:
print(data['purpose_category'].unique()) #выводим уникальные значения столбца purpose_category

['операции с недвижимостью' 'операции с автомобилем'
 'получение образования' 'проведение свадьбы']


Все значения соответствуют заданным в функции. Значение "прочее" не выявлено, то есть все записи были однозначно отнесены к одной из категорий на основе словесного содержания.

Проводится проверка внесенных изменений.

In [40]:
data.info() #смотрим сводку основной информации
data.head() #смотрим как отображаются данные

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21454 non-null  int64 
 1   days_employed          21454 non-null  int64 
 2   dob_years              21454 non-null  int64 
 3   education_id           21454 non-null  int64 
 4   family_status_id       21454 non-null  int64 
 5   gender                 21454 non-null  object
 6   income_type            21454 non-null  object
 7   debt                   21454 non-null  int64 
 8   total_income           21454 non-null  int64 
 9   purpose                21454 non-null  object
 10  total_income_category  21454 non-null  object
 11  purpose_category       21454 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,2194,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


Категоризация целей кредита прошла успешно.

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для ответа на первый вопрос требуется построение сводной таблицы с разбивкой всех записей по числу детей (children) и наличию долга (debt).

In [41]:
data_1 = data.pivot_table(index='children', columns='debt', values='total_income', aggfunc='count') #строим сводную таблицу
data_1['ratio, %'] = data_1[1]/(data_1[1] + data_1[0])*100 #для удобства рассчитываем процент должников
data_1 #смотрим результат

debt,0,1,"ratio, %"
children,,,
0,13028.0,1063.0,7.543822
1,4410.0,445.0,9.165808
2,1926.0,202.0,9.492481
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


##### Вывод 1:

Как показывают данные, наименьший процент должников наблюдается среди людей, не имеющих детей. Это может объясняться меньшим количеством непредвиденных затрат и, как следствие, большей финансовой стабильностью таких клиентов.
При этом среди тех, кто имеет детей, процент должников чуть ниже в случае наличия 3 детей, но данное отклонение скорее всего объясняется относительно небольшим количеством подобных наблюдений. Этим же объясняется полное отсутствие должников среди клиентов с 5 детьми. В целом можно сказать, что сам факт наличия или отсутствия детей (независимо от их количества) влияет на уровень возврата кредитов в срок.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на второй вопрос требуется построение сводной таблицы с разбивкой всех записей по семейному положению (family_status_id) и наличию долга (debt).

In [42]:
data_2 = data.pivot_table(index='family_status_id', columns='debt', values='total_income', aggfunc='count') #строим сводную таблицу
data_2['ratio, %'] = data_2[1]/(data_2[1] + data_2[0])*100 #для удобства рассчитываем процент должников
data_2 = data_2.merge(data_dict_family, on='family_status_id', how='left') #объединяем сводную таблицу со словарем family_status
data_2 #смотрим результат

,family_status_id,0,1,"ratio, %",family_status
0,0,11408,931,7.545182,женат / замужем
1,1,3763,388,9.347145,гражданский брак
2,2,896,63,6.569343,вдовец / вдова
3,3,1110,85,7.112971,в разводе
4,4,2536,274,9.750890,не женат / не замужем


##### Вывод 2:

Соотнесение полученных результатов со словарем семейного положения позволяет сказать, что менее "рискованными" категориями с точки зрения возврата кредита являются женатые/замужние и разведенные люди. Также крайне низкий процент должников наблюдается среди вдовцов и вдов, но это опять-таки может быть обяснено небольшим количеством соответствующих наблюдений. Более "рискованными" категориями, в свою очередь, являются клиенты, которые состоят в гражданском браке или не женатые/не замужние люди.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на третий вопрос требуется построение сводной таблицы с разбивкой всех записей по категории дохода (total_income_category) и наличию долга (debt).

In [43]:
data_3 = data.pivot_table(index='total_income_category', columns='debt', values='total_income', aggfunc='count') #строим сводную таблицу
data_3['ratio, %'] = data_3[1]/(data_3[1] + data_3[0])*100 #для удобства рассчитываем процент должников
data_3 #смотрим результат

debt,0,1,"ratio, %"
total_income_category,,,
A,23,2,8.000000
B,4686,356,7.060690
C,14655,1360,8.492039
D,329,21,6.000000
E,20,2,9.090909


##### Вывод 3:

Значимое количество записей для формирования каких-либо выводов есть только по категориям дохода 'B' и 'C'. Можно предположить, что эти категории в целом задают тенденцию к снижению числа невозвратов кредита при росте дохода, что опять же может объясняться большей финансовой стабильностью и грамотностью клиентов. Однако данных для заключительных выводов в представленной сводке недостаточно. 

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Для ответа на четвертый вопрос требуется построение сводной таблицы с разбивкой всех записей по категории цели кредита (purpose_category) и наличию долга (debt).

In [44]:
data_4 = data.pivot_table(index='purpose_category', columns='debt', values='total_income', aggfunc='count') #строим сводную таблицу
data_4['ratio, %'] = data_4[1]/(data_4[1] + data_4[0])*100 #для удобства рассчитываем процент должников
data_4 #смотрим результат

debt,0,1,"ratio, %"
purpose_category,,,
операции с автомобилем,3903,403,9.359034
операции с недвижимостью,10029,782,7.233373
получение образования,3643,370,9.220035
проведение свадьбы,2138,186,8.003442


##### Вывод 4:

Наблюдается меньший процент должников среди клиентов, берущих кредит на операции с недвижимостью и проведение свадьбы, и больший процент должников среди тех, кто берет кредит на получение образования или операции с автомобилем. Это отчасти может коррелировать с выводами об уровне дохода (операции с недвижимостью обычно более дорогие) и семейном положении (свадьба означает переход в категорию женатых/замужних людей). 

## Общий вывод:

1) Среди клиентов, не имеющих детей, уровень невозврата кредитов на 1-2%* ниже, чем среди клиентов, имеющих хотя бы одного ребенка;  
2) Среди женатых/замужих и разведенных клиентов уровень невозврата кредитов также примерно на 2%* ниже, чем среди клиентов, состоящих в гражданском браке или не женатых/не замужних клиентов;     
3) Уровень дохода и цели кредита также вероятно влияют на степерь возврата кредитов, но для окончательных выводов по данным факторам требуется получение большего объема информации и проведение дополнительного анализа.

'*' - процентные пункты